In [3]:
import os
import re
import csv
import pandas as pd
import numpy as np

# Arbeitsverzeichnis (NIST-Testdaten)
root_dir = r'data/nist'

# Leere Liste für die Daten
data = []

# Gehe durch alle Unterverzeichnisse in NIST
for test_dir in os.listdir(root_dir):
    test_path = os.path.join(root_dir, test_dir)
    
    if os.path.isdir(test_path):
        match = re.search(r'Pi[^_]+', test_dir)
        model_name = match.group(0) if match else "Unknown"

        for subdir, dirs, files in os.walk(test_path):
            if 'PQCsignKAT.time' in files:
                file_path = os.path.join(subdir, 'PQCsignKAT.time')
                algorithm_name = os.path.basename(subdir)

                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.readlines()

                count = None
                prepare = crypto_sig_keypair = crypto_sig_sign = crypto_sig_verify = None

                for line in content:
                    if line.startswith("count ="):
                        if count is not None:
                            data.append([
                                model_name, algorithm_name, count,
                                prepare, crypto_sig_keypair, crypto_sig_sign, crypto_sig_verify
                            ])
                        count = int(line.split('=')[1].strip())

                    if 'prepare (μs)' in line:
                        prepare = int(line.split('=')[1].strip())
                    elif 'crypto_sign_keypair (μs)' in line:
                        crypto_sig_keypair = int(line.split('=')[1].strip())
                    elif 'crypto_sign (μs)' in line:
                        crypto_sig_sign = int(line.split('=')[1].strip())
                    elif 'crypto_sign_open (μs)' in line:
                        crypto_sig_verify = int(line.split('=')[1].strip())

                if count is not None:
                    data.append([
                        model_name, algorithm_name, count,
                        prepare, crypto_sig_keypair, crypto_sig_sign, crypto_sig_verify
                    ])

# DataFrame erzeugen
df = pd.DataFrame(data, columns=[
    'Model', 'Algorithmus', 'count',
    'prepare (μs)', 'keypair (μs)', 'sign (μs)', 'verify (μs)'
])

# Gruppieren und Mittelwerte berechnen
grouped = df.groupby(['Model', 'Algorithmus'])
df_agg = grouped[['keypair (μs)', 'sign (μs)', 'verify (μs)']].mean()
df_count = grouped.size().rename("Runden")

df_combined = df_agg.join(df_count).reset_index()

# Gesamtzeit in μs berechnen
df_combined['Total Time (μs)'] = df_combined[
    ['keypair (μs)', 'sign (μs)', 'verify (μs)']
].sum(axis=1)

# Spaltenreihenfolge
cols = ['Model', 'Algorithmus', 'Runden',
        'keypair (μs)', 'sign (μs)', 'verify (μs)', 'Total Time (μs)']
df_combined = df_combined[cols]

# Sortieren
df_combined.sort_values(by=["Algorithmus", "Model"], inplace=True)

# Export
df_combined.to_csv(
    'output/csv/nist_time_sig.csv',
    index=False,
    float_format='%.0f',
    quoting=csv.QUOTE_NONE
)

print("Export abgeschlossen.")
df_combined


Export abgeschlossen.


,Model,Algorithmus,Runden,keypair (μs),sign (μs),verify (μs),Total Time (μs)
76,Pi3b,CRYSTALS-DILITHIUM_mode2_AES,10,2.305600e+03,7.456600e+03,2356.600000,1.211880e+04
136,Pi4b,CRYSTALS-DILITHIUM_mode2_AES,10,5.069000e+02,1.269800e+03,482.700000,2.259400e+03
270,PiZero2,CRYSTALS-DILITHIUM_mode2_AES,10,1.369200e+03,4.385000e+03,1390.800000,7.145000e+03
77,Pi3b,CRYSTALS-DILITHIUM_mode2_SHA,10,1.190900e+03,9.027900e+03,1403.600000,1.162240e+04
137,Pi4b,CRYSTALS-DILITHIUM_mode2_SHA,10,2.173000e+02,1.304200e+03,233.300000,1.754800e+03
...,...,...,...,...,...,...,...
194,Pi4b,sphincs-shake256-256s-robust,2,6.118245e+05,6.952466e+06,10336.000000,7.574626e+06
268,PiZero,sphincs-shake256-256s-robust,2,1.015432e+07,1.154497e+08,171538.000000,1.257755e+08
135,Pi3b,sphincs-shake256-256s-simple,3,7.766337e+05,6.754087e+06,8776.333333,7.539497e+06
195,Pi4b,sphincs-shake256-256s-simple,3,3.110530e+05,3.698189e+06,5102.000000,4.014344e+06
